# Write SPECFEM3D CMTSOLUTION and STATION files for each EVENT
1. Unpickle events and staions
1. Read and map moment tensors to events
1. Setup and test directory and file names
1. Write CMTSOLUTION and STATION files in separate dictories for each event

#### 1. Unpickle events and stations 

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

#Unpickle events
print('Unpickling Events')
f = open('../pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

#### 2. Read and map moment tensors to events

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Read moment tensors
gf_mts = mtensors('../data/event_moments.csv')
    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

#### 3. Setup and test directory names/paths

In [ ]:
experiments = []
for ekey in e2mt_keys:
    bkeys = dill_straces.getBoreholeKeys()
    bh_stat_list = []
    for bkey in bkeys:
        bh_stat_list.append(dill_straces.getIncludedStations(ekey,bkey))
    bh_stat_tup = tuple(bh_stat_list)   
    exp_set = {'EKEY':ekey,'EVENT':e_cat[ekey],'TENSOR':e2mt_dict[ekey],'STATIONS':bh_stat_tup}
    experiments.append(exp_set)

dir_set_list = []
for eset in experiments:
    fqn = '../SPECFEM3D/events/obs'
    pid = 'EVENT%s' %(str(eset['EKEY']).zfill(3))
    nid = 'NL'
    y = eset['EVENT'].origins[0].time.year
    m = eset['EVENT'].origins[0].time.month
    d = eset['EVENT'].origins[0].time.day
    dname = '%s/%s_%s_%s_%02d_%02d' %(fqn,pid,nid,y,m,d)
    dir_set_list.append((eset,dname))
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    print()

#### 4. Write CMTSOLUTION and STATION files in separate dictories for each event

In [ ]:
from gnam.specutils.write import write_cmtsolution as spec_wcmt
from gnam.specutils.write import write_stations as spec_ws
import os

# define the access rights
access_rights = 0o755

#loop over dir-set pairs
for pair in dir_set_list:
    eset  = pair[0]
    dname = pair[1]
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    try:
        os.makedirs(dname, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % dname)
        print()
    else:
        bkeys = dill_straces.getBoreholeKeys()
        lxy = dill_events.getLocalEventCoord(eset['EKEY'])
        spec_wcmt(dname + '/CMTSOLUTION',eset['TENSOR'],lxy[0],lxy[1])
        spec_ws(dname + '/STATIONS',dill_straces,eset['EKEY'],bkeys)
        print ("Successfully created the directory %s" % dname)
        print()

## Finished